In [1]:
from stylometry import *
import pymongo
import time
import multiprocessing as mp
import pandas as pd
from pandas.io.json import json_normalize
import warnings
warnings.filterwarnings('ignore')

import spacy
#import en_core_web_lg
import en_core_web_md

In [2]:
## connect the database to my virtual machine
client = pymongo.MongoClient('mongodb://localhost/')
db = client['gutenberg_db']
collection = db['gutenberg_collection']

In [3]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'gutenberg_db'), 'gutenberg_collection')

In [5]:
text = collection.find({'file': 'project_gutenberg/51185.txt'}, {'text': 1})[0]['text']

In [6]:
s = time.time()
doc = nlp(text)
tokens = [token.orth_.lower() for token in doc if not token.is_punct and token if not token.is_stop]
sentences = [sent.string.strip() for sent in doc.sents]
print(f'Took {time.time() - s}s to process')

Took 1.6460483074188232s to process


In [7]:
sttr(tokens)

0.627

In [8]:
hapax_legomenon(tokens)

0.486

In [9]:
yules_k(tokens)

1703.7019456860444

In [10]:
function_words(doc)

0.32435685271551074

In [11]:
avg_sentence_length_word(sentences)

8.565289256198348

In [12]:
avg_sentence_length_chars(sentences)

49.239669421487605

In [13]:
avg_syllables_per_word(tokens)

1.7941299790356393

In [14]:
punctuation_sentence(sentences)

2.6743801652892563

In [15]:
shannon_entropy(tokens)

9.51165465635602

In [16]:
simpsons_d(tokens)

0.9968982595359701

In [17]:
average_nps(doc)

221.16666666666666

In [18]:
noun_to_verb(doc)

0.586852207293666

In [19]:
noun_to_adj(doc)

0.8235690235690236

In [20]:
verb_to_adv(doc)

0.6926790024135157

In [21]:
avg_dependency_distance(doc)

2.0988422371735185

In [22]:
get_style_metrics('project_gutenberg/51185.txt')

Took 0.8566100597381592s to process project_gutenberg/51185.txt


### Mongo Upload

In [32]:
filelist = [doc['file'] for doc in collection.find({}, {'file': 1})]
print(len(filelist))

23294


In [11]:
filelist = list(df[df['yules_k'].isnull()].file.values)
print(len(filelist))

3745


In [12]:
for file in filelist[1:]:
    try:
        get_style_metrics(file)
    except (ZeroDivisionError, ValueError) as e: 
        continue

Took 4.671226739883423s to process project_gutenberg/27579.txt
Took 7.935791254043579s to process project_gutenberg/30262.txt
Took 10.151315212249756s to process project_gutenberg/32972.txt
Took 1.950061559677124s to process project_gutenberg/24421.txt
Took 1.3006682395935059s to process project_gutenberg/24420.txt
Took 13.619070291519165s to process project_gutenberg/2442.txt
Took 4.399485349655151s to process project_gutenberg/32970.txt
Took 2.7603251934051514s to process project_gutenberg/3026.txt
Took 12.41840672492981s to process project_gutenberg/27578.txt
Took 1.2692582607269287s to process project_gutenberg/30259.txt
Took 29.45079278945923s to process project_gutenberg/27577-0.txt
Took 7.148463487625122s to process project_gutenberg/32966.txt
Took 5.870404958724976s to process project_gutenberg/30257.txt
Took 14.244499683380127s to process project_gutenberg/27575.txt
Took 7.499291181564331s to process project_gutenberg/32965.txt
Took 2.391129732131958s to process project_gutenb

Took 0.27861642837524414s to process project_gutenberg/24336.txt
Took 1.3092701435089111s to process project_gutenberg/30182.txt
Took 4.993206977844238s to process project_gutenberg/24335.txt
Took 1.4133951663970947s to process project_gutenberg/32905.txt
Took 12.62645173072815s to process project_gutenberg/27501.txt
Took 1.0297050476074219s to process project_gutenberg/32904.txt
Took 1.5942277908325195s to process project_gutenberg/30180.txt
Took 14.335122346878052s to process project_gutenberg/24332.txt
Took 10.696992635726929s to process project_gutenberg/30178.txt
Took 0.18175411224365234s to process project_gutenberg/24331.txt
Took 0.7310609817504883s to process project_gutenberg/32901.txt
Took 0.46943163871765137s to process project_gutenberg/30173.txt
Took 3.3125946521759033s to process project_gutenberg/24330.txt
Took 1.8175644874572754s to process project_gutenberg/275.txt
Took 6.9895548820495605s to process project_gutenberg/329-0.txt
Took 0.2910425662994385s to process proje

Took 15.32371735572815s to process project_gutenberg/27362.txt
Took 0.4040546417236328s to process project_gutenberg/30045.txt
Took 10.812252044677734s to process project_gutenberg/32777.txt
Took 1.3694078922271729s to process project_gutenberg/24186.txt
Took 15.688413381576538s to process project_gutenberg/2736.txt
Took 1.4074604511260986s to process project_gutenberg/32775.txt
Took 4.194713115692139s to process project_gutenberg/30040.txt
Took 12.772725105285645s to process project_gutenberg/24181.txt
Took 6.836271286010742s to process project_gutenberg/32774.txt
Took 12.12573504447937s to process project_gutenberg/27355.txt
Took 12.48343801498413s to process project_gutenberg/24177.txt
Took 0.4141254425048828s to process project_gutenberg/30035.txt
Took 14.255127906799316s to process project_gutenberg/2417.txt
Took 1.9633896350860596s to process project_gutenberg/27353.txt
Took 0.42423439025878906s to process project_gutenberg/30034.txt
Took 7.6425697803497314s to process project_gu

Took 0.45467543601989746s to process project_gutenberg/24083.txt
Took 7.1420207023620605s to process project_gutenberg/2727-0.txt
Took 4.792351484298706s to process project_gutenberg/2996-0.txt
Took 0.6237497329711914s to process project_gutenberg/32716.txt
Took 13.170014142990112s to process project_gutenberg/24078.txt
Took 21.231008052825928s to process project_gutenberg/29958.txt
Took 5.368697881698608s to process project_gutenberg/32715.txt
Took 9.424181461334229s to process project_gutenberg/27268.txt
Took 0.25398969650268555s to process project_gutenberg/29954.txt
Took 0.17265868186950684s to process project_gutenberg/32714.txt
Took 16.44911527633667s to process project_gutenberg/32713.txt
Took 6.537443161010742s to process project_gutenberg/29952.txt
Took 6.748171806335449s to process project_gutenberg/24071.txt
Took 0.5124144554138184s to process project_gutenberg/32711.txt
Took 0.9947586059570312s to process project_gutenberg/24069.txt
Took 3.5809688568115234s to process proje

Took 8.205966711044312s to process project_gutenberg/16077.txt
Took 5.181984186172485s to process project_gutenberg/18829.txt
Took 8.363301038742065s to process project_gutenberg/1315.txt
Took 9.549705982208252s to process project_gutenberg/16076.txt
Took 7.930974721908569s to process project_gutenberg/18828.txt
Took 8.557918548583984s to process project_gutenberg/21750.txt
Took 7.908647775650024s to process project_gutenberg/18828-0.txt
Took 7.29788875579834s to process project_gutenberg/13147.txt
Took 5.164621353149414s to process project_gutenberg/2175.txt
Took 7.496680974960327s to process project_gutenberg/21749.txt
Took 24.55257272720337s to process project_gutenberg/16070.txt
Took 14.297719717025757s to process project_gutenberg/21748.txt
Took 2.240121841430664s to process project_gutenberg/16064.txt
Took 13.956405401229858s to process project_gutenberg/21747.txt
Took 12.182343482971191s to process project_gutenberg/13141.txt
Took 2.53185772895813s to process project_gutenberg/1

...

Took 25.21532416343689s to process project_gutenberg/13983.txt
Took 17.914270401000977s to process project_gutenberg/1957.txt
Took 3.8894314765930176s to process project_gutenberg/11-0.txt
Took 6.227303743362427s to process project_gutenberg/10999.txt
Took 24.477705717086792s to process project_gutenberg/19569.txt
Took 21.033977508544922s to process project_gutenberg/10998.txt
Took 0.7886693477630615s to process project_gutenberg/16580.txt
Took 5.500838041305542s to process project_gutenberg/1658.txt
Took 0.7993311882019043s to process project_gutenberg/19567.txt
Took 14.489887475967407s to process project_gutenberg/10997.txt
Took 9.213874816894531s to process project_gutenberg/16578.txt
Took 20.72443914413452s to process project_gutenberg/13970.txt
Took 13.576799631118774s to process project_gutenberg/19564.txt
Took 17.78126549720764s to process project_gutenberg/10995.txt
Took 0.8246023654937744s to process project_gutenberg/10994.txt
Took 4.624148607254028s to process project_gutenb

Took 11.154057502746582s to process project_gutenberg/10922.txt
Took 4.606462001800537s to process project_gutenberg/13889.txt
Took 18.079619646072388s to process project_gutenberg/16513.txt
Took 15.42526626586914s to process project_gutenberg/10921.txt
Took 8.209091663360596s to process project_gutenberg/16512.txt
Took 2.142665386199951s to process project_gutenberg/13886.txt
Took 18.237995624542236s to process project_gutenberg/10920.txt
Took 21.369192600250244s to process project_gutenberg/16510.txt
Took 16.524279832839966s to process project_gutenberg/1651.txt
Took 2.3272368907928467s to process project_gutenberg/1092-0.txt
Took 16.85936951637268s to process project_gutenberg/13884.txt
Took 15.71006965637207s to process project_gutenberg/13883.txt
Took 3.9395759105682373s to process project_gutenberg/10918.txt
Took 14.804786205291748s to process project_gutenberg/16508.txt
Took 1.1297786235809326s to process project_gutenberg/13881.txt
Took 3.2925944328308105s to process project_gu

Took 4.187394142150879s to process project_gutenberg/10826.txt
Took 9.334553956985474s to process project_gutenberg/13777.txt
Took 19.60677218437195s to process project_gutenberg/16426.txt
Took 0.8718533515930176s to process project_gutenberg/10816.txt
Took 9.02632761001587s to process project_gutenberg/13775.txt
Took 19.19520092010498s to process project_gutenberg/10814.txt
Took 13.842782258987427s to process project_gutenberg/13774.txt
Took 12.679125308990479s to process project_gutenberg/13770.txt
Took 2.024160146713257s to process project_gutenberg/13766.txt
Took 21.834636449813843s to process project_gutenberg/1081-0.txt
Took 12.364977359771729s to process project_gutenberg/13764.txt
Took 6.010178804397583s to process project_gutenberg/16416.txt
Took 18.827130556106567s to process project_gutenberg/13763.txt
Took 0.8788809776306152s to process project_gutenberg/10808.txt
Took 10.660114765167236s to process project_gutenberg/16415.txt
Took 0.6826057434082031s to process project_gut

Took 3.888080358505249s to process project_gutenberg/10731.txt
Took 7.120303153991699s to process project_gutenberg/10729.txt
Took 7.610132455825806s to process project_gutenberg/1632.txt
Took 4.760562181472778s to process project_gutenberg/13652.txt
Took 15.326024532318115s to process project_gutenberg/10727.txt
Took 3.6375153064727783s to process project_gutenberg/10726.txt
Took 4.78002142906189s to process project_gutenberg/10725.txt
Took 6.539049863815308s to process project_gutenberg/10724.txt
Took 51.70062017440796s to process project_gutenberg/1365.txt
Took 3.859041213989258s to process project_gutenberg/10723.txt
Took 0.8183507919311523s to process project_gutenberg/13649.txt
Took 18.06539535522461s to process project_gutenberg/10720.txt
Took 20.606559991836548s to process project_gutenberg/13645.txt
Took 13.332031488418579s to process project_gutenberg/10718.txt
Took 26.1784245967865s to process project_gutenberg/13642.txt
Took 5.949711322784424s to process project_gutenberg/1

KeyboardInterrupt: 

In [10]:
## parallelizing get_style_metrics function
n_cpu = mp.cpu_count()
with mp.Pool(processes = n_cpu) as p:
    p.map(get_style_metrics, filelist)

In [15]:
datapoints = list(collection.find({}, {'_id': 0, 'text': 0}))
df = pd.DataFrame(json_normalize(datapoints))
df = df.rename(columns = {'metrics.sttr': 'sttr', 'metrics.hapax_legomenon': 'hapax_legomenon', 
                          'metrics.yules_k': 'yules_k', 'metrics.function_words': 'function_words', 
                          'metrics.avg_sentence_length_word': 'avg_sentence_length_word',
                          'metrics.avg_sentence_length_chars': 'avg_sentence_length_chars', 
                          'metrics.avg_syllables_per_word': 'avg_syllables_per_word', 
                          'metrics.punctuation_sentence': 'punctuation_sentence', 
                          'metrics.shannon_entropy': 'shannon_entropy',
                          'metrics.simpsons_d': 'simpsons_d', 'metrics.average_nps': 'average_nps', 
                          'metrics.noun_to_verb': 'noun_to_verb', 'metrics.noun_to_adj': 'noun_to_adj', 
                          'metrics.verb_to_adv': 'verb_to_adv', 
                          'metrics.avg_dependency_distance': 'avg_dependency_distance'})
# df.dropna(inplace=True)
print(df.shape)
df.head(10)

(23294, 19)


,file,author,title,year,sttr,hapax_legomenon,yules_k,function_words,avg_sentence_length_word,avg_sentence_length_chars,avg_syllables_per_word,punctuation_sentence,shannon_entropy,simpsons_d,average_nps,noun_to_verb,noun_to_adj,verb_to_adv,avg_dependency_distance
0,project_gutenberg/9999.txt,Sarah H. Bradford,"Harriet, The Moses of Her People",1822,0.624214,0.452214,528.796852,0.398840,20.889773,111.791794,1.735272,4.017759,10.845536,0.998584,232.700000,0.640382,0.792910,0.717273,2.338342
1,project_gutenberg/56195-0.txt,Kenneth Ward,The Boy Volunteers on the Belgian Front,1941,0.585125,0.415000,378.878385,0.397120,14.213115,77.207650,1.752356,2.763206,10.506952,0.997945,232.130435,0.603311,0.811363,0.718462,2.134627
2,project_gutenberg/51187.txt,John Wilson,A Visit to the Mammoth Cave of Kentucky,1972,0.620000,0.444000,1298.919399,0.428477,27.300300,146.405405,1.720949,4.384384,10.003001,0.998360,222.800000,0.656081,0.777569,0.638938,2.403897
3,project_gutenberg/51185.txt,Daniel F. Galouye,All Jackson's Children,1970,0.627000,0.486000,1703.701946,0.324357,8.565289,49.239669,1.794130,2.674380,9.511655,0.996898,221.166667,0.586852,0.823569,0.692679,2.098842
4,project_gutenberg/9997.txt,Francis Parkman,"France and England in North America, Part Third",2013,0.686821,0.541750,269.095482,0.372508,21.562230,122.567266,1.868661,4.204856,11.781284,0.998751,241.057143,0.716877,0.837261,0.751203,2.331626
5,project_gutenberg/9996.txt,Charles Francis Adams,'Tis Sixty Years Since,1814,0.689000,0.525429,1123.982792,0.412379,22.716826,135.894665,2.360974,4.124487,10.880695,0.999149,214.000000,0.671823,0.718001,0.598605,2.411298
6,project_gutenberg/56190-0.txt,Warren Commission,Warren Commission (14 of 26): Hearings Vol. XI...,1963,0.332608,0.195665,212.461619,0.382709,7.211034,37.233989,1.824845,1.806544,9.316746,0.981190,257.088583,0.685171,0.900294,0.686401,2.003238
7,project_gutenberg/9995.txt,Marian M. George,A Little Journey to Puerto Rico,2007,0.644000,0.471333,837.998946,0.409068,14.524590,80.180328,1.754010,2.718460,10.896416,0.998974,246.869565,0.723873,0.794841,0.726814,2.150615
8,project_gutenberg/51183-0.txt,George Grote,"History of Greece, Volume 10 (of 12)",2018,0.678575,0.524642,180.798344,0.350970,15.716207,95.380218,2.143120,2.827180,12.209676,0.999341,245.503968,0.736571,0.807329,0.674299,2.284798
9,project_gutenberg/5619-0.txt,William Petty,Essays on Mankind and Political Arithmetic,1899,0.533643,0.367429,506.564690,0.407927,24.473765,136.352623,1.819653,4.456790,10.645643,0.998158,205.694444,0.707252,0.778920,0.696943,2.465841


In [16]:
df.isnull().sum()

file                           0
author                         0
title                          0
year                           0
sttr                         877
hapax_legomenon              877
yules_k                      414
function_words               414
avg_sentence_length_word     414
avg_sentence_length_chars    414
avg_syllables_per_word       414
punctuation_sentence         414
shannon_entropy              414
simpsons_d                   414
average_nps                  575
noun_to_verb                 414
noun_to_adj                  414
verb_to_adv                  414
avg_dependency_distance      414
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [8]:
df.to_csv('metrics.csv', index=False)